In [12]:
import PyPDF2
import pandas as pd

import re

In [13]:
import nltk
from nltk.corpus import names
import random

nltk.download('names')

[nltk_data] Downloading package names to /Users/mei/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [14]:
# Open the pdf file

PDFfilename = 'frozen.pdf' 

pdfFileObj = open(PDFfilename, 'rb') 

pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

#script = PyPDF2.PdfFileReader(r"/Users/mei/Downloads/frozen.pdf")

#numPages = script.getNumPages()

FileNotFoundError: [Errno 2] No such file or directory: 'frozen.pdf'

In [15]:
# NLTK

def gender_features(word):
    """ feature extractor for the name classifier
    The feature evaluated here is the last letter of a name
    feature name - "last_letter"
    """
    return {"last_letter": word[-1]}  # feature set


if __name__ == "__main__":
    """ Starting block """

    # Extract the data sets
    labeled_names = ([(name, "male") for name in names.words("male.txt")] +
                     [(name, "female") for name in names.words("female.txt")])

In [16]:
print(len(labeled_names))

7944


In [17]:
random.shuffle(labeled_names)

In [18]:
feature_sets = [(gender_features(n), gender)
                    for (n, gender) in labeled_names]

In [19]:
# Divide the feature sets into training and test sets
train_set, test_set = feature_sets[500:], feature_sets[:500]

# Train the naiveBayes classifier
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [59]:
 # Test out the classifier with few samples outside of training set
print(classifier.classify(gender_features("neo")))  # returns male
print(classifier.classify(gender_features("trinity")))  # returns female

male
female
female


In [21]:
print(nltk.classify.accuracy(classifier, test_set))

0.752


In [22]:
print(classifier.show_most_informative_features(5))

Most Informative Features
             last_letter = 'k'              male : female =     44.9 : 1.0
             last_letter = 'a'            female : male   =     35.7 : 1.0
             last_letter = 'f'              male : female =     15.9 : 1.0
             last_letter = 'p'              male : female =     12.5 : 1.0
             last_letter = 'd'              male : female =     10.1 : 1.0
None


In [71]:
# Extract names in scene and add to list

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO

from nameparser.parser import HumanName
from nltk.corpus import wordnet

person_list = []
person_names=person_list
def get_human_names(text):
    tokens = nltk.tokenize.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    sentt = nltk.ne_chunk(pos, binary = False)

    person = []
    name = ""
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
        #if len(person) > 1: #avoid grabbing lone surnames
        for part in person:
            name += part + ' '
            #if name[:-1] not in person_list:
            person_list.append(name[:-1])
            #name = ''
        #person = []

In [72]:
f = open('frozen.txt','r')
text = f.read()

print(text)

INT. CHURCH CHAPEL -- DAY 
 
   Elsa stands at the alter. Anna stands off to one side. She 
   peeks out to the audience. 
 
   Hans waves at her from the pews. He's changed his clothes. 
 
   The crown is placed on Elsa's head. The scepter and orb are 
   presented to Elsa on a pillow. She slowly reaches for them. 
 
                            BISHOP 
                      (a whisper) 
                  Your Majesty, the gloves. 
 
   Elsa hesitates. She breathes nervously, removes her gloves, 
   places them on the pillow. Her hands shake. She takes the orb 
   and scepter, then turns to the people. 
 
                            BISHOP (CONT'D) 
                      (formal, in Old Norse) 
                  Sehm hon HELL-drr IN-um HELL-gum 
                  AYG-num ok krund ee THES-um HELL- 
                  gah STAHTH, ehk teh frahm FUR-ear U- 
                  thear... 
                                                               20 
FROZEN - J. Lee 
 
 
 
   The scepter an

In [102]:
names = get_human_names(text)

for person in person_list:
    person_split = person.split(" ")
    for name in person_split:
        if wordnet.synsets(name):
            #if(name in person_names):
                #person_names.remove(person)
                break

scene_names = person_list[-1].split()

print(scene_names)

f.close()

['Anna', 'Anna', 'Elsa', 'Anna', 'Elsa', 'Elsa', 'Anna', 'Elsa', 'Elsa', 'Elsa']


In [121]:
# Adds one when female name is followed by another female name
def interactions(l):
    counthehe = 0
    countpls = 0
    for i in l:
        if classifier.classify(gender_features(i)) == "female" : 
            if classifier.classify(gender_features(l[counthehe + 1])) == "female" : 
                if i != l[counthehe + 1] :
                    countpls += 1
        counthehe = counthehe + 1
        if counthehe == len(l)-1 :
            break
    return countpls

In [122]:
interactionsCount = interactions(scene_names)

In [123]:
print(interactionsCount)

5


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')